In [9]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from ipywidgets import widgets


/Users/pedrovenancio/opt/anaconda3/envs/ds/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 1. Global Trends in Mental Health Disorder
From Schizophrenia to Depression


## 1. Limpeza

### 1.1. Upload dos dados em bruto

In [10]:
#Carregar o conjunto de dados principal a partir do arquivo CSV especificado ('Mental_health_Depression_disorder_Data.csv') em um DataFrame do pandas
dataset = pd.read_csv('data/raw_data/Mental_health_Depression_disorder_Data.csv')

# Carregar outro conjunto de dados contendo códigos de país ISO a partir do arquivo CSV especificado ('iso_countries.csv') em um DataFrame do pandas
iso_countries = pd.read_csv('data/raw_data/iso_countries.csv')

/var/folders/km/dt_l1qsx1kvdth2cmx91yspm0000gn/T/ipykernel_6257/3986454714.py:2: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('data/raw_data/Mental_health_Depression_disorder_Data.csv')


### 1.2. Criação dos dataframes

In [11]:
# Criar um novo DataFrame do pandas chamado 'df_geral' e preenchê-lo com os dados do conjunto de dados principal
df_geral = pd.DataFrame(dataset)

# Criar um novo DataFrame do pandas chamado 'df_iso' e preenchê-lo com os dados do conjunto de dados de códigos de país ISO
df_iso = pd.DataFrame(iso_countries)

### 1.3. Identificação das sub-tabelas

In [12]:
# Criar uma visualização de subconjunto (subtable_view) do DataFrame df_geral, selecionando apenas as linhas em que a coluna 'Year' é igual à string 'Year' para identificar os cabeçalhos de coluna
subtable_view = df_geral[df_geral['Year'] == 'Year']

# Exibir as primeiras linhas da visualização de subconjunto usando o método head(), que por padrão mostra as primeiras 5 linhas
subtable_view.head()


,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
6468,6468,Entity,Code,Year,Prevalence in males (%),Prevalence in females (%),Population,NaN,NaN,NaN,NaN
54276,54276,Entity,Code,Year,"Suicide rate (deaths per 100,000 individuals)",Depressive disorder rates (number suffering pe...,Population,NaN,NaN,NaN,NaN
102084,102084,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both ...,NaN,NaN,NaN,NaN,NaN,NaN


### 1.4. Identificação dos índices para separação das tabelas

In [13]:
# Criar uma lista (index_lst) contendo os valores da coluna 'index' da visualização de subconjunto (subtable_view)
index_lst = subtable_view['index'].tolist()

# Inserir o valor 0 no início da lista
index_lst.insert(0, 0)

# Adicionar o último valor da coluna 'index' do DataFrame original (df_geral) ao final da lista
index_lst.append(df_geral['index'].iloc[-1])

# Imprimir a lista resultante
print(index_lst)


[0, 6468, 54276, 102084, 108552]


### 1.5. Operações de limpeza do dataframe principal e criação de novos dataframes a partir das sub-tabelas

In [14]:
# Criar um dicionário vazio para armazenar os DataFrames resultantes
dfs = {}

# Iterar sobre os elementos da lista de índices
for i in range(len(index_lst) - 1):
    # Definir o início e o final do intervalo com base nos valores da lista de índices
    start = index_lst[i]
    end = index_lst[i + 1] - 1  # Subtrai 1 para obter o intervalo correto

    # Criar um novo DataFrame (dfs[f'df_{i}']) com base no intervalo definido
    dfs[f'df_{i}'] = df_geral.iloc[start:end + 1].copy()

    # Resetar os índices e remover colunas que são compostas apenas por valores nulos (NaN)
    dfs[f'df_{i}'].reset_index(drop=True, inplace=True)
    dfs[f'df_{i}'] = dfs[f'df_{i}'].dropna(axis=1, how='all')
    
    # Substituir 'NaN' na coluna 'Code' pelo valor correspondente na coluna 'Entity'
    dfs[f'df_{i}']['Code'].fillna(dfs[f'df_{i}']['Entity'], inplace=True)

    # Remover linhas com valores nulos nas colunas
    dfs[f'df_{i}'] = dfs[f'df_{i}'].dropna(subset=dfs[f'df_{i}'].columns)

    # Renomear as colunas com base na primeira linha, exceto para o primeiro DataFrame
    if i > 0:
        col_names = dfs[f'df_{i}'].iloc[0].tolist()
        dfs[f'df_{i}'].columns = col_names

    # Remover a linha que foi usada como nome de colunas
    dfs[f'df_{i}'] = dfs[f'df_{i}'].iloc[1:].reset_index(drop=True)
    
    # Filtrar linhas onde 'Code' não está presente em df_iso['alpha-3']
    dfs[f'sub_df_{i}'] = dfs[f'df_{i}'][~dfs[f'df_{i}']['Code'].isin(df_iso['alpha-3'].str.upper())].reset_index(drop=True)
    
    # Fazer drop da coluna 'Code' em dfs[f'sub_df_{i}'] porque é igual a 'Entity'
    dfs[f'sub_df_{i}'] = dfs[f'sub_df_{i}'].drop(columns=['Code'])
    
    # Filtrar linhas onde 'Code' está presente em df_iso['alpha-3'] e guardar num novo DataFrame
    dfs[f'df_{i}'] = dfs[f'df_{i}'][dfs[f'df_{i}']['Code'].isin(df_iso['alpha-3'])].reset_index(drop=True)
    
    # Guardar o DataFrame como um arquivo CSV com um nome correspondente ao índice
    dfs[f'df_{i}'].to_csv(f'data/clean_data/df_{i}.csv', index=False)
    dfs[f'sub_df_{i}'].to_csv(f'data/clean_data/sub_df_{i}.csv', index=False)


## 2. Análise

### 2.1. Gráficos

#### Mapa

#### Gráfico a

#### Grafico b

#### Gráfico c

#### Gráfico d

In [15]:
europe = [alpha3 for alpha3, region in zip(df_iso['alpha-3'], df_iso['region']) if region == 'Europe']


In [16]:
df_iso['region'].unique()

array(['Asia', 'Europe', 'Africa', 'Oceania', 'Americas', nan],
      dtype=object)

In [17]:
data_df_0 = pd.read_csv('data/clean_data/df_0.csv')
df_0 = pd.DataFrame(data_df_0)
df_0_2000 = df_0[(df_0['Year'] == 2000) & (df_0['Code'].isin(europe))]

df_0_2000.head()

,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
37,38,Albania,ALB,2000,0.194541,0.698437,0.132704,3.360267,0.492401,2.195285,1.654338
121,150,Andorra,AND,2000,0.263767,0.968843,0.609424,5.281196,0.876375,3.751057,1.218826
289,346,Austria,AUT,2000,0.256413,0.947524,0.608221,5.362011,0.846521,3.473469,1.899831
457,514,Belarus,BLR,2000,0.189298,0.693549,0.159122,2.919701,0.751862,4.181462,5.301741
485,542,Belgium,BEL,2000,0.259447,0.960924,0.551042,5.231671,0.847280,3.786139,1.481386


In [18]:
data_df_2 = pd.read_csv('data/clean_data/df_2.csv')
df_2 = pd.DataFrame(data_df_2)
df_2_2000 = df_2[(df_2['Year'] == 2000) & (df_2['Code'].isin(europe))]

df_2_2000.head()

,54276,Entity,Code,Year,"Suicide rate (deaths per 100,000 individuals)","Depressive disorder rates (number suffering per 100,000)",Population
38,54819,Albania,ALB,2000,4.867020,2113.579055,3129000.0
122,55357,Andorra,AND,2000,9.533035,3444.080321,65000.0
290,56967,Austria,AUT,2000,15.916630,3188.891969,8069000.0
458,58287,Belarus,BLR,2000,33.727580,3939.177028,9872000.0
486,58507,Belgium,BEL,2000,18.497711,3420.780043,10282000.0


In [19]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

# Suponha que df_0 seja o DataFrame que contém os dados
# Certifique-se de substituir 'Anxiety disorders (%)' pelo nome correto da coluna que deseja usar inicialmente.

# Função para atualizar o gráfico com base na condição selecionada
def update_map(change):
    selected_condition = change.new
    fig.data[0].z = df_2[selected_condition]
    fig.update_layout(
        title=f"World map of {selected_condition} over years",
        coloraxis=dict(cmin=df_2[selected_condition].min(), cmax=df_2[selected_condition].max())
    )


# Suponha que df_0 seja o DataFrame que contém os dados
# Certifique-se de substituir 'Anxiety disorders (%)' pelo nome correto da coluna que deseja usar inicialmente.
options = df_2.columns[4:].tolist()  # Exclui as colunas 0 a 4
condition_dropdown = widgets.Dropdown(
    description='Condition:   ',
    value=options[0],
    options=options
)

# Crie o gráfico inicial
fig = px.choropleth(df_2, locations="Code",
                    color=condition_dropdown.value,
                    scope="world",
                    hover_name="Entity",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    animation_frame="Year", 
                    animation_group="Entity")

# Customize layout
fig.update_layout(
    title=f"World map of {condition_dropdown.value} over years",  # Modificado de initial_condition para condition_dropdown.value
    font=dict(family="Arial", size=12),
    margin=dict(l=5, r=5, t=50, b=5),
)

fig.update_geos(
    showframe=False,
)
fig.update_layout(
    title=f"World map of {condition_dropdown.value} over years",
    font=dict(family="Arial", size=12),
    margin=dict(l=5, r=5, t=50, b=5),
    coloraxis=dict(cmin=df_2[condition_dropdown.value].min(), cmax=df_2[condition_dropdown.value].max())
)


# Use a função de retorno de chamada para atualizar o gráfico quando a opção do seletor for alterada
condition_dropdown.observe(update_map, names='value')

# Exiba o gráfico e o seletor
display(condition_dropdown)
display(fig)


Dropdown(description='Condition:   ', options=('Suicide rate (deaths per 100,000 individuals)', 'Depressive di…

### 2.2. Tabelas

### Menu

In [ ]:
def menu():
    print('0 - Terminar\n1 - Obter DataFrame\n2 - Gráficos\n3 - Tabelas\n4 - x\n')
    opt1 = input('Insira uma opção: ')
    if opt1 == "0":
        return False
        print('Opção 0')
    elif opt1 == "1":
        print('Opção 1')
    elif opt1 == "2":
        print('Gráficos')           
    elif opt1 == "3":
        print('Opção 3')       
    elif opt1 == "4":
        print('Opção 4')      
    else:
        print('Opção inválida. A opção deve estar entre 0 e 4.')

### Função principal

In [ ]:
def main():
    while True:
        menu()
        
main()